In [7]:
# this code parses a draw.io-generated XML tree to extract the children elements and create a full text doc.
#  This is useful if you're a fan of structured thinking (like a mindmap) but then want to have the text easily output

# notes: 
#   * clearly there is a lot of refactoring possible, this is merely a test drive to see if it's a useful tool
#   * the top of the doc has to be structured like <?xml version="1.0"?> <root> (i.e. have to swap out the 'graph'
#   * it only prints out the elements without children (i.e. the lowest granularity), except if the element is bolded (it'll then include that)
#  
#   To Do:  write this to Google Sheets and back (i.e. allow easy iteration of outline and text)    


In [ ]:
import xml.etree.ElementTree as ET
import re
import functools
import os

tree = ET.parse('map.xml')
root_cal = tree.getroot()
root = root_cal.findall(".")

/Users/matthowes/Drive/Data Dev and Viz/Projects/book_from_mindmap


In [9]:
number_of_elements = len(root[0])
text_array = [] 
relationship_array = []
for i in range(0, number_of_elements):
   title_state =""
   label =""
   entry_type = 'unknown'
   individual_cell = root[0][i]
   individual_cell_attributes = individual_cell.attrib 
   #print (individual_cell_attributes)
  
   if 'style' in individual_cell_attributes:
      text_string =  root[0][i].attrib['style']
      
      if "fontStyle=1" in  text_string:
         title_state = "title"
         print (text_string)  
      else: 
        title_state = "no_title"
   if 'id' in individual_cell_attributes:
      id =root[0][i].attrib['id']
     # print ('id:' +  id )
   if 'label' in individual_cell_attributes:
      label =root[0][i].attrib['label']
     # print ('label:' +  label )    
    
   if 'treeRoot' in individual_cell_attributes:
      treeRoot =root[0][i].attrib['treeRoot']
     # print ('treeroot:' +  treeRoot )        
      parent_node = id
   if 'value' in individual_cell_attributes:
      val =root[0][i].attrib['value']
      if len(val) > 0:  
          entry_type = 'text'
          label = val  
      else:
          entry_type = 'geo' 
   else: 
      entry_type ="unknown"
   if 'source' in individual_cell_attributes:
      source =root[0][i].attrib['source']
     # print ('source:' + val) 
   if 'target' in individual_cell_attributes:
      target =root[0][i].attrib['target']
      #print ('target:' + val)
   geometry_within_cell= root[0][i].findall('mxGeometry')
   for child in geometry_within_cell:
       if 'y' in  child.attrib:
           y  = child.attrib['y']
      #     print ('y:' +  y )
           
   if entry_type == 'geo':
      relationship_array.append([id,source, target])
   elif entry_type == 'text':
      text_array.append([id, label,title_state,y])
   source= ""  
   target= "" 
   id = "" 

#print (parent_node)


whiteSpace=wrap;html=1;rounded=1;arcSize=50;align=center;verticalAlign=middle;container=1;recursiveResize=0;strokeWidth=1;autosize=1;spacing=4;treeFolding=1;fontStyle=1
whiteSpace=wrap;html=1;rounded=1;arcSize=50;align=center;verticalAlign=middle;container=1;recursiveResize=0;strokeWidth=1;autosize=1;spacing=4;treeFolding=1;fontStyle=1


In [10]:
for row in text_array: 
     print (row)


['O3UHbDxIHgIvfHBMtrS7-7', 'Looking at individual rows of data', 'no_title', '255']
['O3UHbDxIHgIvfHBMtrS7-8', 'Looking at max, mins and other statistics&nbsp;', 'no_title', '295']
['O3UHbDxIHgIvfHBMtrS7-11', 'Abscombe quartet', 'no_title', '320']
['O3UHbDxIHgIvfHBMtrS7-13', 'Graphing in Tableau or other viz tools&nbsp;', 'no_title', '366']
['O3UHbDxIHgIvfHBMtrS7-16', 'Complex, but less complex than looking at the raw data.&nbsp; Incremental tool, not savior', 'no_title', '658.5']
['O3UHbDxIHgIvfHBMtrS7-18', '3D viz enables that exploration&nbsp;', 'no_title', '511']
['O3UHbDxIHgIvfHBMtrS7-23', 'How to use&nbsp;', 'no_title', '824']
['O3UHbDxIHgIvfHBMtrS7-27', 'Using some Javascript libraries, I built a quick tool to parse and view "many-to-many" datasets and show them in three dimensions. (Many-to-many datasets might include:&nbsp; transactions between countries, social networks, or website traffic.)', 'no_title', '423.5']
['O3UHbDxIHgIvfHBMtrS7-29', '<div style="text-align: left"><sp

In [11]:
#for row in relationship_array: 
 #    print (row)

In [13]:
merged_array = []
for row in  text_array:  
    id_name = row[0]
    text_val = row[1]
    title_val = row[2]
    y_val = row[3]

    parent_search= next((x for x in relationship_array if x[2] == id_name ), 'none')
    parent_name = parent_search[1]

    child_search = next((x for x in relationship_array if x[1] == id_name ), 'none')
 
    if child_search =="none" : 
       has_children = "false"
    else: 
       has_children = "true" 
    merged_array.append([ id_name,parent_name, has_children ,y_val,title_val, text_val ])

In [14]:
def sortSecond(val): 
   # print  (val[3])
    return int(float(val[3]))  
    
merged_array.sort(key = sortSecond, reverse=True )     
#for row in merged_array: 
 #    print (row)

In [15]:

repls =  (('&gt;', '>'),
          ('&lt;', '<'),
          ('&nbsp;', ''),
          ('<div style="text-align: left">',''),
          ('<span>',  ''),
          ('</span>',  ''),
          ('</div>',  ''),)

def replace_all(text, dic):
    for i, j in dic.iteritems():
        text = text.replace(i, j)
    return text

In [16]:
for row in merged_array: 
     print (row)

['O3UHbDxIHgIvfHBMtrS7-37', 'gHReWdxfJyFHyzUZcx94-1', 'false', '1300', 'no_title', 'I\'ll continue to evolve this tool and other projects.&nbsp; Feel free to hit me up on &lt;a href="https://www.linkedin.com/in/matthowes/"&gt;LinkedIn&lt;/a&gt;.']
['O3UHbDxIHgIvfHBMtrS7-26', 'O3UHbDxIHgIvfHBMtrS7-34', 'false', '1208.5', 'no_title', 'There is also great tooling available in Python.&nbsp;']
['O3UHbDxIHgIvfHBMtrS7-34', 'gHReWdxfJyFHyzUZcx94-1', 'true', '1155.5', 'title', 'Other tools']
['O3UHbDxIHgIvfHBMtrS7-25', 'O3UHbDxIHgIvfHBMtrS7-34', 'false', '1155.5', 'no_title', 'Since this is built with Javascript, there are dozens of other frameworks that can probably show a similar view.&nbsp;']
['O3UHbDxIHgIvfHBMtrS7-35', 'O3UHbDxIHgIvfHBMtrS7-45', 'false', '1070', 'no_title', '<div style="text-align: left"><span>When hosted in a standalone website, this visualization can be embedded in Tableau or other tool.&nbsp; This would help tell the story more comprehensively.&nbsp;</span></div>']
['O3U

In [20]:
master_string =[]

def recursive_print (target_name):
   #print ("target name: " + target_name)
   # look up the entry value 

   find_entry = next((x for x in merged_array if x[0] == target_name ), 'none')
   
   
   if find_entry == 'none': # than this is the parent 
       has_children ='true' 
      
   else:     
       label_value = find_entry[5]    
       has_children = find_entry[2]
       title_value = find_entry[4] 
       if (title_value == 'title'):
          master_string.append(" " + label_value)
         # print (label_value)
   if (has_children == 'true'):
     
        # find all children and sort by y location 
        #print (target_name)   
        matches = [x for x in merged_array if x[1] == target_name ]    
        matches.sort(key = sortSecond, reverse=False )  
       
  
        
        #iterate over them in order 
        for child_name in matches: 
           # print (child_name[0])
            recursive_print(child_name[0])
   else:
       for r in repls:
          label_value = label_value.replace(*r)
       master_string.append(" " + label_value)
       #print (label_value)                     
                             

recursive_print(parent_node)

temp =""

for x in master_string: 
    temp += x +"\n\n"

    
print (temp)  





 Tableau, PowerBI and other tools are great for some data visualization, but I wanted to use some newer tools to better visualize multi-dimensional data.

 Using some Javascript libraries, I built a quick tool to parse and view "many-to-many" datasets and show them in three dimensions. (Many-to-many datasets might include: transactions between countries, social networks, or website traffic.)

 The "creator" version of this tool is used to parse and filter raw datasets. It isn't ready for public release, but I wanted to show some data in the "viewer" version.

 You can see that this view represents a lot of complex data in its entirety. This isn't meant to be something that I drop into an executive's Powerpoint and walk away.  This is more of a working tool that I can use to use to explore the data. Any insights I discover would be distilled into more accessible line charts or other visualizations.<br>

 Be sure to click around the viz and zoom around.

 You can click the "rotation" but

In [ ]:
with open("filename.txt", "w") as f:
    f.write(temp)
